In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
import transformers
from finetune_peft import get_peft_config, PEFTArguments
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig, set_peft_model_state_dict
import peft

from peft.tuners.lora import Linear
import torch.nn.functional as F
from peft.utils.other import transpose

import time

/home/ubuntu/minimal-llama/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_path = "/home/ubuntu/llama-weights/7B/llama-7b"
tokenizer_path = model_path

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = transformers.LlamaForCausalLM.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 33/33 [00:10<00:00,  3.13it/s]


In [ ]:
# torch.view vs torch.reshape

In [114]:
b, l, c = 2, 10, 1024
r = 8

X1 = torch.randn(l, c)
X2 = torch.randn(l, c)
X = torch.cat([X1, X2], dim=0)
XX = torch.cat([X1, X2], dim=1)

In [115]:
A1 = torch.randn(c, r)
A2 = torch.randn(c, r)
A = torch.cat([A1, A2], dim=1)
Y = X @ A

In [116]:
AA = torch.cat([torch.cat([A1, torch.zeros_like(A1)]), torch.cat([torch.zeros_like(A2), A2])], dim=1)

In [117]:
(XX @ AA)[:,r:] - X2 @ A2

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [68]:
print(Y.shape)
Y1 = Y[:l, :r]
Y2 = Y[l:, r:]

torch.Size([20, 16])


In [69]:
(Y1 - X1 @ A1).abs()

tensor([[0.0000, 0.0039, 0.0000, 0.0000, 0.0156, 0.0000, 0.0000, 0.0312],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0156, 0.0156],
        [0.0000, 0.0000, 0.0024, 0.0000, 0.0039, 0.0078, 0.0078, 0.0000],
        [0.0000, 0.0078, 0.0000, 0.0078, 0.0078, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0098, 0.0000, 0.0156, 0.0312, 0.0000, 0.0000, 0.0000],
        [0.0156, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0156, 0.0000, 0.0000, 0.0156, 0.0000],
        [0.0078, 0.0000, 0.0039, 0.0156, 0.0000, 0.0000, 0.0020, 0.0000],
        [0.0000, 0.0000, 0.0068, 0.0078, 0.0000, 0.0000, 0.0000, 0.0312],
        [0.0000, 0.0000, 0.0039, 0.0039, 0.0000, 0.0000, 0.0000, 0.0117]])

In [ ]:
model = PeftModel.from_pretrained(model, "jondurbin/airoboros-7b-gpt4-1.2-peft", adapter_name="lora1")
model = PeftModel.from_pretrained(model.base_model.model, "jondurbin/airoboros-7b-gpt4-1.2-peft", adapter_name="lora2")
# model = PeftModel.from_pretrained(model.base_model.model, "trl-lib/llama-7b-se-rl-peft", adapter_name="se-rl")

In [ ]:
model.base_model.model.model.layers[0].self_attn.q_proj

In [ ]:
type(list(model.base_model.model.model.layers[0].self_attn.q_proj.named_modules())[0][1])

In [ ]:
for name, module in model.named_modules():
    if isinstance(module, peft.tuners.lora.Linear):
        module
        break
        # Do whatever operation you need to perform here
        # For example, if you want to add lora_A and lora_B to the parameter, you can do:
        # parameter.data = parameter.data + self.lora_A + self.lora_B

In [ ]:
lora_a_merged = torch.nn.Linear(module.lora_A['lora1'].in_features, module.lora_A['lora1'].out_features)
# module.lora_A['merged']

In [ ]:
from peft.tuners.lora import Linear

def forward(self, x: torch.Tensor):
    previous_dtype = x.dtype
    if self.active_adapter not in self.lora_A.keys():
        return F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)
    if self.disable_adapters:
        if self.r[self.active_adapter] > 0 and self.merged:
            self.unmerge()
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)
    elif self.r[self.active_adapter] > 0 and not self.merged:
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)

        x = x.to(self.lora_A[self.active_adapter].weight.dtype)

        result += (
            self.lora_B[self.active_adapter](
                self.lora_A[self.active_adapter](self.lora_dropout[self.active_adapter](x))
            )
            * self.scaling[self.active_adapter]
        )
    else:
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)

    result = result.to(previous_dtype)

    return result


Linear.forward = forward

In [ ]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)

tokenizer = transformers.LlamaTokenizer.from_pretrained(tokenizer_path)
batch = tokenizer("The LLaMA language model is", return_tensors="pt")

with torch.no_grad():
    out1 = model.generate(
        input_ids=batch["input_ids"],
        attention_mask=torch.ones_like(batch["input_ids"]),
        max_length=200,
    )
print(tokenizer.decode(out1[0]))

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)

In [ ]:
bs = 2
seq_len = 10
ctx_dim = 4096
rank = 8
scaling = 4.0
fan_in_fan_out = False
bias = None
device = 'cuda:0'

In [ ]:
# blora
x0 = torch.randn([bs, seq_len, ctx_dim], device=device)
x1 = torch.randn([bs, seq_len, ctx_dim], device=device)
weight = torch.randn(([ctx_dim, ctx_dim]), device=device)

lora_a0 = torch.nn.Linear(in_features=ctx_dim, out_features=rank, bias=False, device=device)
lora_b0 = torch.nn.Linear(in_features=rank, out_features=ctx_dim, bias=False, device=device)

lora_a1 = torch.nn.Linear(in_features=ctx_dim, out_features=rank, bias=False, device=device)
lora_b1 = torch.nn.Linear(in_features=rank, out_features=ctx_dim, bias=False, device=device)

loraa = torch.cat([lora_a0.weight, lora_a1.weight], dim=1)
lorab = torch.cat([lora_b0.weight, lora_b1.weight], dim=0)

# forward pass
def lora_forward(x, weight, lora1, lora2, scaling):
    result = F.linear(x0, transpose(weight, fan_in_fan_out), bias=bias)

    if isinstance(lora1, torch.nn.Linear):
        x = x.to(lora1.weight.dtype)
        result += scaling * lora2(lora1(x))
        return result
    else:
        x = x.reshape(seq_len, -1)
        x = x.to(lora1.dtype)
        out = F.linear(x, transpose(lora1, fan_in_fan_out), bias=bias)
        out = scaling * F.linear(out, transpose(lora2, fan_in_fan_out), bias=bias)
        out = out.reshape(bs, seq_len, -1)
        result += out
    return result

start = time.time()
r0 = lora_forward(x0, weight, lora_a0, lora_b0, scaling)
r1 = lora_forward(x1, weight, lora_a1, lora_b1, scaling)
print(f"lora_forward: {(time.time() - start)*1e6} microseceonds")

In [ ]:
lora_a0.weight.shape

In [ ]:
lora_b0.weight.shape

In [ ]:
class BLora(torch.nn.Module):
    def __init__(
        self,
        lora1: list,
        lora2: list,
        weight: torch.Tensor,
    ):
        super().__init__()
        self.lora_a = torch.nn.Parameter(torch.cat([lora1[0].weight, lora2[0].weight], dim=1))
        self.lora_b = torch.nn.Parameter(torch.cat([lora1[1].weight, lora2[1].weight], dim=0))
        self.weight = torch.nn.Parameter(weight)

    def forward(self, x: torch.Tensor):
        result = F.linear(x, transpose(self.weight, fan_in_fan_out), bias=bias)
        x = x.reshape(seq_len, -1)
        x = x.to(self.lora_a.dtype)

        out = F.linear(x, transpose(self.lora_a, fan_in_fan_out), bias=bias)
        out = scaling * F.linear(out, transpose(self.lora_b, fan_in_fan_out), bias=bias)
        out = out.reshape(bs, seq_len, -1)
        result += out
        return result
    
blora = BLora(lora1=[lora_a0, lora_b0], lora2=[lora_a1, lora_b1], weight=weight)

start = time.time()
r2 = blora(x0)
print(f"lora_forward: {(time.time() - start)*1e6} microseceonds")

In [ ]:
r0

In [ ]:
r1

In [ ]:
r2[1]

In [ ]:
def forward(self, x: torch.Tensor):
    previous_dtype = x.dtype
    if self.active_adapter not in self.lora_A.keys():
        return F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)
    if self.disable_adapters:
        if self.r[self.active_adapter] > 0 and self.merged:
            self.unmerge()
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)
    elif self.r[self.active_adapter] > 0 and not self.merged:
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)

        x = x.to(self.lora_A[self.active_adapter].weight.dtype)

        result += (
            self.lora_B[self.active_adapter](
                self.lora_A[self.active_adapter](self.lora_dropout[self.active_adapter](x))
            )
            * self.scaling[self.active_adapter]
        )
    else:
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)

    result = result.to(previous_dtype)

    return result

In [ ]:
# depreciated blora
x0 = torch.randn([bs, seq_len, ctx_dim])
weight = torch.randn(([ctx_dim, ctx_dim]))

lora_a0 = torch.nn.Linear(in_features=ctx_dim, out_features=rank, bias=False)
lora_b0 = torch.nn.Linear(in_features=rank, out_features=ctx_dim, bias=False)

lora_a1 = torch.nn.Linear(in_features=ctx_dim, out_features=rank, bias=False)
lora_b1 = torch.nn.Linear(in_features=rank, out_features=ctx_dim, bias=False)

lora1 = torch.nn.Linear(in_features=bs * ctx_dim, out_features=rank, bias=False)
lora2 = torch.nn.Linear(in_features=rank, out_features=bs * ctx_dim, bias=False)

lora1.weight = torch.nn.Parameter(torch.cat([lora_a0.weight, lora_a1.weight], dim=1))
lora2.weight = torch.nn.Parameter(torch.cat([lora_b0.weight, lora_b1.weight], dim=0))

# forward pass
result1 = F.linear(x0, transpose(weight, fan_in_fan_out), bias=bias)
x1 = x0.reshape(seq_len, -1)
x1 = x1.to(lora1.weight.dtype)
out1 = lora2(lora1(x1)) * scaling
out1 = out1.reshape(bs, seq_len, -1)
result1 += out1

In [ ]:
path = "/home/ubuntu/airoboros-7b-gpt4-1.2-peft/adapter_model.bin"
peft_model_state_dict = torch.load(path)

peft_config_path = "/home/ubuntu/airoboros-7b-gpt4-1.2-peft/adapter_config.json"
peft_config = PeftConfig.from_json_file(peft_config_path)
peft_config = {'default' : peft_config}
model.peft_config = peft_config
set_peft_model_state_dict(model, peft_model_state_dict, adapter_name="default")